<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/myProspects_ATH_TTM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2025-05-11 13:23:12


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/quantvesting/data/myProspectsScrips.csv')

mypf = mypf[mypf['LatestQtr'] == 0]
#mypf = mypf[mypf['v80'] == 1]

stock_n100 = mypf['Symbol'].unique()

tmp_df = mypf
list_x40 = list(tmp_df[tmp_df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
list_x5k = list(tmp_df[tmp_df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
list_x40n = list(tmp_df[tmp_df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
list_hc = list(tmp_df[tmp_df['Conviction'].isin(['X-LC', 'H-LC', 'X-MC', 'X-SC'])]['Symbol'].values)
#len(set(list_x40 + list_x5k) - set( list_x40n))
stock_list = list(set(list_x40 + list_x5k + list_hc))

exclude = ['MCDOWELL-N', 'SCHAEFFLER ', 'IBREALEST', 'NESTLEIND']

stock_n100 = stock_list

#stock_ids

stock_ids = stock_n100[:]
start_year = 'Mar 2020'
curr_qtr = 'Mar 2025' # 'Dec 2023'
curr_year = 'Mar 2025'
print('Stocks to be scanned:',len(stock_ids))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Stocks to be scanned: 54


In [4]:
stock_ids = sorted(stock_ids, reverse=True)

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids:

  print(stock_name)

  link_blueprint = "https://www.screener.in/company/"

  stock_list = ['3MINDIA', 'AUBANK', 'COLPAL', 'GLAND', 'LUXIND', 'WHIRLPOOL' , 'IEX', 'ROSSARI',  'TATAELXSI',
                 'EQUITAS', 'AAVAS', 'BIRLAMONEY', 'ABBOTINDIA', 'AKZOINDIA', 'ALKYLAMINE','ARE&M', 'SURYODAY']

  actual_link = link_blueprint + stock_name.upper()   #+ "/consolidated/"

  def get_list(actual_link):
      page = requests.get(actual_link)
      link_soup = BeautifulSoup(page.content, 'html.parser')

      try:
        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')
      except ValueError:
        print(stock_name + ' => No Test List')
      pass  # do nothing!

      while("" in test_list) :
          test_list.remove("")
      while("\r" in test_list) :
          test_list.remove("\r")
      while(" \r" in test_list) :
          test_list.remove(" \r")

      return test_list
  #print(actual_link)
  test_list = get_list(actual_link)

  def get_yoy(test_list):
    #print(len(test_list))
    n = len(test_list)

    if n < 100:
        actual_link = link_blueprint + stock_name.upper()
        test_list = get_list(actual_link)
        #print(len(test_list))

    #print(test_list)
    if 'TTM' not in test_list:
        end_year = curr_year
    else:
        end_year = 'TTM'

    df_yoy = pd.DataFrame()

    print(test_list.index(end_year))
    s = test_list.index(end_year) +1 - test_list.index(start_year)

    df_yoy['Year'] = test_list[test_list.index(start_year):test_list.index(end_year)+1]
    df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
    df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

    df_yoy.set_index('Year', inplace=True)
    df_yoy = df_yoy.T
    df_yoy['Stock'] = stock_name.upper()
    #df_yoy
    return df_yoy
  page = requests.get(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')
  scan_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')
  #print(scan_list.index('Sep 2023'))
  try:
    scan_list.index(curr_qtr) > 30
    df_latest_analysis = pd.concat([df_latest_analysis, get_yoy(test_list)])
  except ValueError:
      print(stock_name + ' => No ' + curr_qtr)
      pass  # do nothing!

WHIRLPOOL
WHIRLPOOL => No Mar 2025
VIPIND
VIPIND => No Mar 2025
VINATIORGA
VINATIORGA => No Mar 2025
UNITDSPR
UNITDSPR => No Mar 2025
TTKPRESTIG
TTKPRESTIG => No Mar 2025
TEAMLEASE
TEAMLEASE => No Mar 2025
TATAMOTORS


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
print('Stocks updated:', df_latest_analysis.shape[0]/2)
if df_latest_analysis.shape[0] > 0:
  lastestq_stocks = list(df_latest_analysis['Stock'].unique())
  len(lastestq_stocks)

In [ ]:
if df_latest_analysis.shape[0] > 0:
  #df_analysis.columns
  df_latest_analysis = df_latest_analysis.reset_index()
  df_latest_analysis.rename({'Mar 2025': 'Latest', 'index': 'Type'}, axis=1, inplace=True)
  cols = [ 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', 'Mar 2024', 'Latest'] # 'Latest'
  #df_analysis[cols] = df_analysis[cols].apply(pd.to_numeric)
  for col in cols:
      df_latest_analysis[col] = df_latest_analysis[col].str.replace(',', '').astype(float)

In [ ]:
cols = df_latest_analysis.columns[1:-1]
if df_latest_analysis.shape[0] > 0:
  df_latest_analysis['Max'] = df_latest_analysis[cols].max(axis=1)
  df_latest_analysis['Diff%'] = round((df_latest_analysis['Max'] - df_latest_analysis['Latest'])*100/df_latest_analysis['Max'],2) # 'Latest'

In [ ]:
if df_latest_analysis.shape[0] > 0:
  df_master_analysis = pd.read_csv('/content/drive/My Drive/quantvesting/data/myProspects-ATH-TTM.csv')
  df_master_analysis.columns = df_latest_analysis.columns
  df_master_analysis = df_master_analysis[~df_master_analysis['Stock'].isin(lastestq_stocks)]
  df_master_analysis = pd.concat([df_master_analysis, df_latest_analysis])
  df_master_analysis.to_csv('/content/drive/My Drive/quantvesting/data/myProspects-ATH-TTM.csv', index=False)

In [ ]:
if df_latest_analysis.shape[0] > 0:
  df_tmp = df_latest_analysis[df_latest_analysis['Diff%'] <= 5]
  tmp = pd.DataFrame(df_tmp['Stock'].value_counts())
  len(tmp[tmp['count']>1].index)
  star_stocks = list(tmp[tmp['count']>1].index)
  print(len(star_stocks))

In [ ]:
mypf = pd.read_csv('/content/drive/My Drive/quantvesting/data/myProspectsScrips.csv')
mypf.loc[mypf['Symbol'].isin(lastestq_stocks), 'LatestQtr'] = 1
mypf.loc[mypf['Symbol'].isin(lastestq_stocks), 'StarStock'] = 0
mypf.loc[mypf['Symbol'].isin(star_stocks), 'StarStock'] = 1
mypf.to_csv('/content/drive/My Drive/quantvesting/data/myProspectsScrips.csv', index=False)